In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample

In [ ]:
df = pd.read_csv("./raw_dataset.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
def isBad(row):
  sum = row["toxic"] + row["severe_toxic"] + row["obscene"] + row["threat"] + row["insult"] + row["identity_hate"]
  return 1 if sum > 1 else 0

df["label"] = df.apply(isBad, axis=1)

In [ ]:
df = df.drop(columns=["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])

In [ ]:
df.head()

In [ ]:
df['label'].value_counts()

In [ ]:
df_0 = df[df.label == 0]
df_1 = df[df.label == 1]

samples_per_class = 4000

df_0_downsampled = resample(df_0, replace=False, n_samples=samples_per_class, random_state=42)
df_1_downsampled = resample(df_1, replace=False, n_samples=samples_per_class, random_state=42)
df = pd.concat([df_0_downsampled, df_1_downsampled])

In [ ]:
df['label'].value_counts()

In [ ]:
class BadWordClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, 
            return_tensors='pt', 
            padding='max_length', 
            truncation=True, 
            max_length=self.max_length
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
bert_model_name = "bert-base-cased"
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["comment_text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = BadWordClassificationDataset(train_texts, train_labels, tokenizer, max_length)
test_dataset = BadWordClassificationDataset(test_texts, test_labels, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
class BERTBadWordClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTBadWordClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_built() else "cuda" if torch.cuda.is_available() else "cpu")

model = BERTBadWordClassifier(bert_model_name, num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, data_loader, criterion, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions, target_names=['Not Bad', 'Bad'])

In [ ]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    epoch_loss = train(model, train_dataloader, criterion, optimizer, scheduler, device)
    accuracy, report = evaluate(model, test_dataloader, device)
    print(f'Loss: {epoch_loss:.4f}')
    print(f'Validation Accuracy: {accuracy:.4f}')
    print(report)

In [ ]:
torch.save(model.state_dict(), './classifier_4000.pth')